In [484]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import openpyxl


In [ ]:
! jupyter notebook --NotebookApp.iopub_data_rate_limit=2147483647

[I 09:24:20.233 NotebookApp] The port 8888 is already in use, trying another port.
[I 09:24:20.253 NotebookApp] [jupyter_nbextensions_configurator] enabled 0.4.1
[I 09:24:20.253 NotebookApp] Serving notebooks from local directory: /Users/siruozhang/Desktop
[I 09:24:20.253 NotebookApp] The Jupyter Notebook is running at:
[I 09:24:20.253 NotebookApp] http://localhost:8889/?token=f5a9d43e5f6967ebfadc25bdfc8a9b02833f50cd4256f878
[I 09:24:20.253 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 09:24:20.259 NotebookApp] 
    
    To access the notebook, open this file in a browser:
        file:///Users/siruozhang/Library/Jupyter/runtime/nbserver-10259-open.html
    Or copy and paste one of these URLs:
        http://localhost:8889/?token=f5a9d43e5f6967ebfadc25bdfc8a9b02833f50cd4256f878


In [2]:
file_name = "/Users/siruozhang/Desktop/dkdata.json"
f_obj = open(file_name)
data = json.load(f_obj)
col_names=data['data'][1].keys()

In [3]:
lista=[]
for i in col_names:
    lista.append(i)

In [4]:
print(lista)

['payer_name', 'payer_jurisdiction', 'payer_account', 'source_file', 'amount_orig', 'id', 'beneficiary_type', 'beneficiary_core', 'amount_orig_currency', 'beneficiary_name', 'beneficiary_jurisdiction', 'investigation', 'beneficiary_bank_country', 'beneficiary_name_norm', 'payer_core', 'beneficiary_account', 'purpose', 'date', 'amount_usd', 'amount_eur', 'payer_type', 'payer_name_norm', 'payer_bank_country']


In [529]:
df = pd.DataFrame(data['data'],columns=lista)  
df.head()
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16940 entries, 0 to 16939
Data columns (total 23 columns):
payer_name                  16940 non-null object
payer_jurisdiction          16940 non-null object
payer_account               16825 non-null object
source_file                 16940 non-null object
amount_orig                 16940 non-null float64
id                          16940 non-null int64
beneficiary_type            16940 non-null object
beneficiary_core            16940 non-null bool
amount_orig_currency        16940 non-null object
beneficiary_name            16940 non-null object
beneficiary_jurisdiction    16940 non-null object
investigation               16940 non-null object
beneficiary_bank_country    16936 non-null object
beneficiary_name_norm       16940 non-null object
payer_core                  16940 non-null bool
beneficiary_account         16936 non-null object
purpose                     16940 non-null object
date                        16940 non-null ob

In [6]:
df['eur']=df['amount_eur'].str.replace('$','').str.replace(',','').astype(float)

In [7]:
df['amount_usd']/df['eur']
df1 = df.drop('amount_eur',axis=1)

In [8]:
#df2 = df1.dropna(subset=['beneficiary_account','beneficiary_bank_country'],axis=0)


In [9]:
miss_value_index={}
for i in ['payer_account','payer_bank_country','beneficiary_account','beneficiary_bank_country']:
    miss_value_index[i]=df1[df1[i].isna()].index.tolist()
miss_value_index['payer_account']==miss_value_index['payer_bank_country']
miss_value_index['beneficiary_account']==miss_value_index['beneficiary_bank_country']
print(miss_value_index)


{'payer_account': [67, 121, 308, 451, 509, 603, 1210, 1211, 2923, 3084, 3508, 3510, 3677, 4229, 4441, 4823, 5073, 5124, 5126, 5590, 6232, 6234, 6236, 6237, 6385, 6493, 6636, 6643, 6646, 6703, 6733, 7277, 7279, 7449, 7704, 7785, 7786, 7789, 7793, 7796, 7799, 7803, 7807, 7836, 7837, 7838, 7842, 7844, 7845, 7850, 7892, 7898, 7899, 7901, 7938, 8060, 8061, 8062, 8063, 8137, 8144, 8275, 8479, 8694, 9094, 9095, 9385, 9481, 9606, 9704, 9929, 9930, 9931, 9935, 10121, 10123, 10944, 11286, 11382, 11820, 11821, 11824, 11825, 11925, 11926, 12014, 12160, 12383, 12572, 12599, 12829, 12872, 12873, 12900, 12901, 12940, 12941, 12943, 12946, 12948, 12949, 12974, 12975, 13274, 13536, 14156, 14317, 15343, 15613, 15993, 16089, 16171, 16285, 16288, 16692], 'payer_bank_country': [67, 121, 308, 451, 509, 603, 1210, 1211, 2923, 3084, 3508, 3510, 3677, 4229, 4441, 4823, 5073, 5124, 5126, 5590, 6232, 6234, 6236, 6237, 6385, 6493, 6636, 6643, 6646, 6703, 6733, 7277, 7279, 7449, 7704, 7785, 7786, 7789, 7793, 7796, 

In [10]:
df1.iloc[miss_value_index['beneficiary_bank_country']]

,payer_name,payer_jurisdiction,payer_account,source_file,amount_orig,id,beneficiary_type,beneficiary_core,amount_orig_currency,beneficiary_name,...,beneficiary_name_norm,payer_core,beneficiary_account,purpose,date,amount_usd,payer_type,payer_name_norm,payer_bank_country,eur
306,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-1800.0,10738,Company,False,USD,CONTINENTAL ENGINES,...,CONTINENTAL ENGINES,True,None,1208105026359433 ORDER NUMBER NO 1179550 DD 18...,2012-08-10,1800,Company,METASTAR INVEST LLP,EE,1468.05
3598,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-2909.5,15956,Company,False,USD,MICROCOPY,...,MICROCOPY,True,None,1302265026564645 INVOICE SO12000014 DD 13/12/2...,2013-02-26,2910,Company,METASTAR INVEST LLP,EE,2131.43
8782,METASTAR INVEST LLP,GB,EE77 3300 3334 8704 0004,pdf/METASTAR Account statement 30.06.12-31.12....,-4311.0,16287,Company,False,USD,MICROCOPY,...,MICROCOPY,True,None,1307175022068793 INVOICE SOM1300027 DD 08/07/2...,2013-07-17,4311,Company,METASTAR INVEST LLP,EE,3301.41
11302,HILUX SERVICES LP,GB,EE12 3300 3335 1615 0001,pdf/HILUX Allegato 4.xml,-253150.0,4658,Company,False,USD,TURBILLION LLC,...,TURBILLION LLC,True,None,1311125026365614 PAYMENT TO THE CONTRACT N 121...,2013-11-12,253150,Company,HILUX SERVICES LP,EE,186918.37


In [11]:
np.unique(df1[df1['payer_core']==True]['payer_name'].values)
df1[df1['payer_core']==True]['payer_name'].value_counts()

HILUX SERVICES LP                      5178
METASTAR INVEST LLP                    3841
LCM ALLIANCE LLP                       2820
POLUX MANAGEMENT LP                    1344
METASTAR INVEST LLP CORNWALL BUILD.       5
Name: payer_name, dtype: int64

In [12]:
df1['payer_jurisdiction'].value_counts()
df1['payer_type'].value_counts()

Company    16877
Invalid       51
Person        12
Name: payer_type, dtype: int64

In [13]:
df1['payer_name_new']=df1['payer_name'].str.replace('.','').str.upper()
df1['payer_name_norm_new']=df1['payer_name_norm'].str.replace('.','').str.upper()

In [14]:
payer_name_not_same1=[]
for i in range(len(df1['payer_name_new'])):
    if df1['payer_name_new'].iloc[i] in df1['payer_name_norm_new'].iloc[i]:
        pass
    else:
        payer_name_not_same1.append(i)
payer_name_not_same2=[]

for i in range(len(df1['payer_name_new'])):
    if df1['payer_name_norm_new'].iloc[i] in df1['payer_name_new'].iloc[i]:
        pass
    else:
        payer_name_not_same2.append(i)
        

intersection= [val for val in payer_name_not_same1 if val in payer_name_not_same2]

intersection


[37,
 40,
 59,
 85,
 105,
 122,
 139,
 149,
 164,
 165,
 166,
 203,
 319,
 420,
 439,
 542,
 550,
 551,
 609,
 628,
 651,
 786,
 807,
 879,
 903,
 1059,
 1217,
 1223,
 1224,
 1243,
 1247,
 1281,
 1284,
 1336,
 1454,
 1468,
 1471,
 1525,
 1598,
 1601,
 1671,
 1699,
 1700,
 1718,
 1721,
 1745,
 1794,
 1827,
 1853,
 1854,
 1977,
 2032,
 2060,
 2096,
 2097,
 2116,
 2149,
 2362,
 2374,
 2414,
 2479,
 2511,
 2517,
 2544,
 2580,
 2584,
 2585,
 2588,
 2685,
 2733,
 2737,
 2780,
 2786,
 2787,
 2788,
 2837,
 2850,
 2861,
 2881,
 2884,
 2885,
 2900,
 2924,
 2958,
 3048,
 3124,
 3129,
 3132,
 3160,
 3185,
 3186,
 3190,
 3191,
 3194,
 3195,
 3196,
 3197,
 3221,
 3222,
 3251,
 3277,
 3297,
 3429,
 3648,
 3687,
 3752,
 4233,
 4307,
 4373,
 4393,
 4420,
 4445,
 4479,
 4481,
 4482,
 4532,
 4618,
 4658,
 4665,
 4666,
 4668,
 4719,
 4727,
 4728,
 4952,
 5000,
 5066,
 5076,
 5139,
 5140,
 5194,
 5202,
 5203,
 5252,
 5254,
 5255,
 5484,
 5585,
 5674,
 6017,
 6042,
 6228,
 6229,
 6321,
 6326,
 6391,
 6496,


In [15]:
def doubt_object_payer_name(n):
    dic={}
    for i in intersection:
        if list(df1['payer_name_norm_new'].iloc[i])[:n] != list(df1['payer_name_new'].iloc[i])[:n]:
            dic[i]=[df1['payer_name_norm_new'].iloc[i],df1['payer_name_new'].iloc[i]]
    return(dic)

a=[]
for k in range(20):
    for i in doubt_object_payer_name(k).keys():
        a.append(doubt_object_payer_name(k)[i])
    print(k,np.unique(a))


0 []
1 ['OJSC ARCELORMITTAL KRYVIY RIH' 'PJSC ARCELORMITTAL KRYVYI RIH,']
2 ['OJSC ARCELORMITTAL KRYVIY RIH' 'PJSC ARCELORMITTAL KRYVYI RIH,']
3 ['AL GHARABALLY NATIONAL TRADING' 'ALGHARABALLY NATIONAL TRADING'
 'APOLLO(CHINA) CO,LTD' 'APPOLLO (CHINA) CO, LTD'
 'OJSC ARCELORMITTAL KRYVIY RIH' 'PJSC ARCELORMITTAL KRYVYI RIH,'
 'ZI WU TRADING CO LIMITED' 'ZI YI TRADING CO LIMITED'
 'ZIWU TRADING CO LIMITED' 'ZIWU TRADING CO LIMITED NO285 LO'
 'ZIWU TRADING CO LIMITED RM289 14/F' 'ZIYI TRADING CO LIMITED']
4 ['AL GHARABALLY NATIONAL TRADING' 'ALGHARABALLY NATIONAL TRADING'
 'APOLLO(CHINA) CO,LTD' 'APPOLLO (CHINA) CO, LTD'
 'HUA HE TRADE CO LIMITED' 'HUA HE TRADE CO LIMITED RM289 14/F'
 'HUA MENG TRADE CO, LTD' 'HUAHE TRADE CO LIMITED'
 'HUAMENG TRADE CO LIMITED' 'HUAMENG TRADE CO LIMITED RM289 14/F'
 'HUAMENG TRADECO LIMITEDRM289 14/F G' 'OJSC ARCELORMITTAL KRYVIY RIH'
 'PJSC ARCELORMITTAL KRYVYI RIH,' 'ZI WU TRADING CO LIMITED'
 'ZI YI TRADING CO LIMITED' 'ZIWU TRADING CO LIMITED'
 'ZIWU

15 ['AL GHARABALLY NATIONAL TRADING' 'ALGHARABALLY NATIONAL TRADING'
 'APOLLO(CHINA) CO,LTD' 'APPOLLO (CHINA) CO, LTD' 'ASTON GLOBAL L P'
 'ASTON GLOBAL LP' 'DANBELL  IMPORT  LLP' 'DANBELL IMPORT LLP'
 'GENTECH (FZE)' 'GENTECH FZE' 'HANSE TRADEING CO LIMITED'
 'HANSE TRADING CO LIMITED' 'HUA HE TRADE CO LIMITED'
 'HUA HE TRADE CO LIMITED RM289 14/F' 'HUA MENG TRADE CO, LTD'
 'HUAHE TRADE CO LIMITED' 'HUAMENG TRADE CO LIMITED'
 'HUAMENG TRADE CO LIMITED RM289 14/F'
 'HUAMENG TRADECO LIMITEDRM289 14/F G' 'LUXBORG L P' 'LUXBORG LP'
 'OJSC ARCELORMITTAL KRYVIY RIH' 'PJSC ARCELORMITTAL KRYVYI RIH,'
 'STILL HAIR ACCESSORY CO, LIMITED' 'STILL HAIRA ACCESSORY CO LIMITED'
 "TOO 'ARMAS GROUP'" 'TOO ARMAS GROUP' 'VOSTOK MEDIA ECHANGE LTD'
 'VOSTOK MEDIA EXCHANGE LTD' 'YW THEHAAM CO LIMITED'
 'YW THEHAAM CO, LIMITED' 'YW THEHAAM CO,LIMITED'
 'ZI WU TRADING CO LIMITED' 'ZI YI TRADING CO LIMITED'
 'ZIWU TRADING CO LIMITED' 'ZIWU TRADING CO LIMITED NO285 LO'
 'ZIWU TRADING CO LIMITED RM289 14/F' 'ZIY

In [16]:
print(df1['payer_bank_country'].value_counts())
print(df1['payer_jurisdiction'].value_counts())



EE    13147
33     2083
AZ      562
LV      270
KZ      201
CY      135
40       72
NR       67
OS       50
CH       37
NL       30
AE       26
20       20
10       16
TR       12
60       11
LI        8
38        7
GE        7
00        6
DE        5
MT        4
TX        4
06        4
TA        4
39        3
CZ        3
12        2
LT        2
41        2
IT        2
01        2
MD        2
36        2
90        2
DK        2
BG        1
55        1
35        1
26        1
66        1
BE        1
11        1
LU        1
RO        1
50        1
87        1
QA        1
FR        1
Name: payer_bank_country, dtype: int64
GB         15074
AZ           609
UNKNOWN      205
KZ           201
NZ           134
CY           132
CN           108
RU            76
HK            71
DE            41
US            40
CA            37
VG            34
AE            33
TR            26
PA            19
ES            16
EE            15
FI            11
MT             9
GE             7
SG             4

In [17]:
a1=[]
for i in range(len(df1['beneficiary_bank_country'])):
    if df1['beneficiary_bank_country'].iloc[i] is not None:
        if len(list(df1['beneficiary_bank_country'].iloc[i])) != 2:
            a1.append(i)
print(a1)
#df['beneficiary_bank_country'].value_counts()
#all benficiary bank country have 2 character except null.

a2=[]
for i in range(len(df1['payer_bank_country'])):
    if df1['payer_bank_country'].iloc[i] is not None:
        if len(list(df1['payer_bank_country'].iloc[i])) != 2:
            a1.append(i)
print(a2)
#all payer bank country have 2 character except null.

[]
[]


In [18]:
a3=[]
for i in range(len(df1['beneficiary_bank_country'])):
    if (df1['beneficiary_bank_country'].iloc[i] is not None) and (df1['beneficiary_account'].iloc[i] is not None):
        if (list(df1['beneficiary_bank_country'].iloc[i])[:2]) != (list(df1['beneficiary_account'].iloc[i])[:2]):
            a3.append(i)
print(a3)


a4=[]
for i in range(len(df1['payer_bank_country'])):
    if (df1['payer_bank_country'].iloc[i] is not None) and (df1['payer_account'].iloc[i] is not None):
        if (list(df1['payer_bank_country'].iloc[i])[:2]) != (list(df1['payer_account'].iloc[i])[:2]):
            a4.append(i)
print(a4)
#_bank country info can get from account



[]
[]


In [113]:
#df1[df1['amount_orig']<0]['payer_jurisdiction'].value_counts()
#df1[(df1['amount_orig']>0) & (df1['payer_jurisdiction']=='GB')]
#df1['beneficiary_bank_country'].value_counts()
#df1[df1['amount_orig']<0]['beneficiary_core'].value_counts()
#df1[df1['payer_core']==True]['payer_name'].value_counts()
#df1[df1['beneficiary_core']==True]['beneficiary_name'].value_counts()



a4=[]
for i in range(len(df1['source_file'])):
    a4.append(re.findall(r'/(.*) [Account|Allegato]',df1['source_file'].iloc[i])[0])
a12=[]
a6=[]
for i in range(len(df1['amount_orig'])):
    if df1['amount_orig'].iloc[i]<0:
        a6.append(i)

In [51]:
pd.set_option('display.max_columns', None)

In [63]:
a5=[]
for i in a6:
        if a4[i][0] not in df1['payer_name_norm'].iloc[i]:
            a5.append(i)
len(a5)
# amount<0, sources names belong to payer_name
a61=[]
for i in a6:
    a61.append(a4[i][0])
print(np.unique(a61))
df1['payer_name_norm'][(df1['payer_core']==True)].value_counts()

['HILUX' 'LCM ALLIANCE' 'METASTAR' 'POLUX']


HILUX SERVICES LP      5178
METASTAR INVEST LLP    3846
LCM ALLIANCE LLP       2820
POLUX MANAGEMENT LP    1344
Name: payer_name_norm, dtype: int64

In [68]:
a51=[]
for i in range(len(df1['source_file'])):
    if df1['amount_orig'].iloc[i]>=0:
        if a4[i][0] not in df1['beneficiary_name_norm'].iloc[i]:
            a51.append(i)
len(a51)

aa=[]

for i in range(len('source_file')):
    if df1['payer_name_norm'].iloc[i] == df1['beneficiary_name_norm'].iloc[i]:
        aa.append(i)
print(aa)
#no same company at payer and bene
# amount>0, sources name belong to bene_name 

#Thus, I think amount +/- does not effect on data

[]


In [132]:
currency_detect=[]
for i in range(len(df1['amount_orig'])):
    if df1['amount_orig_currency'].iloc[i]=='USD':
        if df1['amount_usd'].iloc[i]!= math.ceil(abs(df1['amount_orig'].iloc[i])):
            currency_detect.append(i)
    if df1['amount_orig_currency'].iloc[i]=='EUR':
        if math.ceil(df1['eur'].iloc[i]) != math.ceil(abs(df1['amount_orig'].iloc[i])):
            currency_detect.append(i)
print(currency_detect)
df1['amount_orig_currency'].value_counts()
# I think keep usd is safe
#investigation only one level,thus drop
df2=df1.drop(['amount_orig','eur','amount_orig_currency','payer_name','payer_name_new','investigation','beneficiary_name','source_file'],axis=1)
df2.info()

[]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16940 entries, 0 to 16939
Data columns (total 17 columns):
payer_jurisdiction          16940 non-null object
payer_account               16825 non-null object
id                          16940 non-null int64
beneficiary_type            16940 non-null object
beneficiary_core            16940 non-null bool
beneficiary_jurisdiction    16940 non-null object
beneficiary_bank_country    16936 non-null object
beneficiary_name_norm       16940 non-null object
payer_core                  16940 non-null bool
beneficiary_account         16936 non-null object
purpose                     16940 non-null object
date                        16940 non-null object
amount_usd                  16940 non-null int64
payer_type                  16940 non-null object
payer_name_norm             16940 non-null object
payer_bank_country          16825 non-null object
payer_name_norm_new         16940 non-null object
dtypes: bool(2), int64(2), object(13)
memory

In [133]:
date_=[]
for i in range(len(df1['source_file'])):
    date_.append(re.findall(r'\d*.\d*.\d*-\d*.\d*.\d*',df1['source_file'].iloc[0])[0])
np.unique(date_)
#all number look like date are same
df2['source']=a4
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16940 entries, 0 to 16939
Data columns (total 18 columns):
payer_jurisdiction          16940 non-null object
payer_account               16825 non-null object
id                          16940 non-null int64
beneficiary_type            16940 non-null object
beneficiary_core            16940 non-null bool
beneficiary_jurisdiction    16940 non-null object
beneficiary_bank_country    16936 non-null object
beneficiary_name_norm       16940 non-null object
payer_core                  16940 non-null bool
beneficiary_account         16936 non-null object
purpose                     16940 non-null object
date                        16940 non-null object
amount_usd                  16940 non-null int64
payer_type                  16940 non-null object
payer_name_norm             16940 non-null object
payer_bank_country          16825 non-null object
payer_name_norm_new         16940 non-null object
source                      16940 non-null obje

In [134]:
df2

,payer_jurisdiction,payer_account,id,beneficiary_type,beneficiary_core,beneficiary_jurisdiction,beneficiary_bank_country,beneficiary_name_norm,payer_core,beneficiary_account,purpose,date,amount_usd,payer_type,payer_name_norm,payer_bank_country,payer_name_norm_new,source
0,AZ,33817018409333311204,6049,Company,True,GB,EE,LCM ALLIANCE LLP,False,EE27 3300 3335 0561 0002,1206295100052180 OCT4121800021 ADVANCE PAYM FO...,2012-06-30,535470,Company,AZARBAYCAN METANOL KOMPANI MMC,33,AZARBAYCAN METANOL KOMPANI MMC,LCM ALLIANCE
1,GB,EE27 3300 3335 0561 0002,6050,Company,False,GB,33,MOBILA LLP,True,333504500003,1207035026699176 INVOICE.No 6215 DD 25.05.2012,2012-07-03,535000,Company,LCM ALLIANCE LLP,EE,LCM ALLIANCE LLP,LCM ALLIANCE
2,GB,20100374548222,10623,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,"1207065103089249 /FEE/USD4,81",2012-07-06,90536,Company,SKN ELECTRICAL SERVICES LIMITED,20,SKN ELECTRICAL SERVICES LIMITED,METASTAR
3,GB,EE77 3300 3334 8704 0004,15589,Company,False,VG,33,INMAXO CAPITAL CORP.,True,333455870002,1207095022358525 DOGOVOR ZAYMA,2012-07-09,90520,Company,METASTAR INVEST LLP,EE,METASTAR INVEST LLP,METASTAR
4,GB,EE77 3300 3334 8704 0004,10624,Company,False,VG,33,INMAXO CAPITAL CORP.,True,333455870002,1207135024578077 DOGOVOR ZAYMA,2012-07-13,60,Company,METASTAR INVEST LLP,EE,METASTAR INVEST LLP,METASTAR
5,VG,333455870002,15030,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,CONTRACT N 001 DD 24.12.2009,2012-07-13,239099,Company,INMAXO CAPITAL CORP.,33,INMAXO CAPITAL CORP,METASTAR
6,VG,333455870002,15590,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1207135026233261 CONTRACT N 001 DD 24.12.2009,2012-07-13,225850,Company,INMAXO CAPITAL CORP.,33,INMAXO CAPITAL CORP,METASTAR
7,GB,EE77 3300 3334 8704 0004,9314,Company,False,NZ,33,JETFIELD NETWORKS LIMITED,True,333488110001,invoice 888 dd 09.06.2012,2012-07-16,130313,Company,METASTAR INVEST LLP,EE,METASTAR INVEST LLP,METASTAR
8,GB,EE77 3300 3334 8704 0004,9313,Company,False,IT,IT,ERREBI SPA,True,IT78T0605569721000000001000,1207165020172345 INVOICE 8456 DD 10.07.2012,2012-07-16,118696,Company,METASTAR INVEST LLP,EE,METASTAR INVEST LLP,METASTAR
9,GB,EE77 3300 3334 8704 0004,15595,Company,False,GB,33,LOTA SALES LLP,True,333484540002,1207165025935425 DOGOVOR ZAYMA,2012-07-16,80000,Company,METASTAR INVEST LLP,EE,METASTAR INVEST LLP,METASTAR


In [160]:
df2[['purpose','beneficiary_account','payer_account']][(df2['payer_type']=='Invalid')|(df2['payer_type']=='Person')]



,payer_jurisdiction,payer_account,id,beneficiary_type,beneficiary_core,beneficiary_jurisdiction,beneficiary_bank_country,beneficiary_name_norm,payer_core,beneficiary_account,purpose,date,amount_usd,payer_type,payer_name_norm,payer_bank_country,payer_name_norm_new,source
67,DE,None,9327,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1207185100280220 RTN 1207135025890645/7033/ 13...,2012-07-19,167676,Invalid,DEUTDEFFXXX,None,DEUTDEFFXXX,METASTAR
121,CN,None,10669,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1207245104516412 345587 INMAXO CAPITAL CORP./...,2012-07-24,49963,Invalid,BKCHCNBJ,None,BKCHCNBJ,METASTAR
308,DE,None,9386,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1208095105222495 RETURN OF P/O 159/12073050228...,2012-08-10,81992,Invalid,DEUTDEFFXXX,None,DEUTDEFFXXX,METASTAR
451,TR,None,10778,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1208275104667672 RTN P/O 1207265020271628/132...,2012-08-27,36920,Invalid,DENITRIS,None,DENITRIS,METASTAR
476,TR,TR500006200009100009097093,10785,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1208285105855742 /RFB/GOODS VALUE,2012-08-28,67955,Person,SEMRA UZUNCIMEN,TR,SEMRA UZUNCIMEN,METASTAR
509,US,None,15673,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1208305106394756 RTN P/O 1208225027994320/301...,2012-08-30,4000,Invalid,TDOMUS33,None,TDOMUS33,METASTAR
603,GB,None,10581,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1209055105244285 RTN P/O 1208315022336943/361...,2012-09-05,15872,Invalid,BARCGB22,None,BARCGB22,METASTAR
1211,DE,None,15136,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1210125104899072 RTN P/O 1210105026740299/638...,2012-10-12,1601,Invalid,NORSDE71,None,NORSDE71,METASTAR
1679,CZ,CZ7355000000004054712001,5432,Company,True,GB,EE,LCM ALLIANCE LLP,False,EE27 3300 3335 0561 0002,1211095103858410 31000121109E00040010000001 EB...,2012-11-09,71043,Person,RASHAD GAFAROV,CZ,RASHAD GAFAROV,LCM ALLIANCE
2583,AZ,TXID/AZ/1300959901,11223,Company,True,GB,EE,METASTAR INVEST LLP,False,EE77 3300 3334 8704 0004,1212245101390337 PREPAYMENT DUE TO CONT.08/11 ...,2012-12-27,149455,Person,AZERB.REP.MINIST.EMERGENCY SITUA,TX,AZERBREPMINISTEMERGENCY SITUA,METASTAR


In [147]:
df['beneficiary_type'].value_counts()

Company    15998
Person       932
Invalid       10
Name: beneficiary_type, dtype: int64

In [170]:
df2['payer_account']
df2['payer_bank_country']
country=[]
for i in range(len(df2['payer_account'])):
    if (df2['payer_account'].iloc[i]!=None) & (df2['payer_bank_country'].iloc[i]!=None):
        if (list(df2['payer_account'].iloc[i])[:2])!=(list(df2['payer_bank_country'].iloc[i])[:2]):
            country.append(i)
country

country1=[]
for i in range(len(df2['beneficiary_account'])):
    if (df2['beneficiary_account'].iloc[i]!=None) & (df2['beneficiary_bank_country'].iloc[i]!=None):
        if (list(df2['beneficiary_account'].iloc[i])[:2])!=(list(df2['beneficiary_bank_country'].iloc[i])[:2]):
            country1.append(i)
country1

##account first 2 letter == bank country if not none

[]

In [244]:
pd.options.mode.chained_assignment = None
df2['payer_account']=df['payer_account']
df2['beneficiary_account']=df['beneficiary_account']
#try
df2['beneficiary_account'].iloc[0]=''.join(list(df2['beneficiary_account'].iloc[0][2:]))



In [245]:
for i in range(len(df2['payer_account'])):
    if (df2['payer_account'].iloc[i]!=None) :
        df2['payer_account'].iloc[i]=''.join(list(df2['payer_account'].iloc[i])[2:])
df2['payer_account']=df2['payer_account'].str.replace(' ','')


for i in range(1,len(df2['beneficiary_account'])):
    if (df2['beneficiary_account'].iloc[i]!=None):
        df2['beneficiary_account'].iloc[i]=''.join(list(df2['beneficiary_account'].iloc[i])[2:])
df2['beneficiary_account']=df2['beneficiary_account'].str.replace(' ','')
df2['beneficiary_account']
#delete bank country at first two letter and drop space

0               273300333505610002
1                       3504500003
2               773300333487040004
3                       3455870002
4                       3455870002
5               773300333487040004
6               773300333487040004
7                       3488110001
8        78T0605569721000000001000
9                       3484540002
10                      3484540002
11                      3499570009
12        410012300615100016777301
13       78H0306920409100000012385
14        070013400000000039500042
15                      3499570009
16        240006200034100009092916
17            96810700000210018800
18                      3468790001
19              773300333487040004
20              773300333487040004
21              773300333487040004
22              702840722002001727
23           710200000025597675100
24                      3488110001
25                      3488110001
26                      3488110001
27        480006400000212780107701
28            114004

In [255]:
df2['purpose']

0        1206295100052180 OCT4121800021 ADVANCE PAYM FO...
1           1207035026699176 INVOICE.No 6215 DD 25.05.2012
2                          1207065103089249   /FEE/USD4,81
3                           1207095022358525 DOGOVOR ZAYMA
4                           1207135024578077 DOGOVOR ZAYMA
5                             CONTRACT N 001 DD 24.12.2009
6            1207135026233261 CONTRACT N 001 DD 24.12.2009
7                                invoice 888 dd 09.06.2012
8              1207165020172345 INVOICE 8456 DD 10.07.2012
9                           1207165025935425 DOGOVOR ZAYMA
10                          1207165022969353 DOGOVOR ZAYMA
11              1207165028140593 INVOICE 666 DD 06.06.2012
12              1207165020742469 INVOICE 743 DD 10.06.2012
13              1207165026491333 INVOICE 2-9 DD 11.07.2012
14                     1207165022275789 AHMADOV ANAR SUBAR
15              1207165021570793 INVOICE 666 DD 06.06.2012
16              1207165022255697 INVOICE 785 DD 13.07.20

In [552]:
for i in range(len(df2['purpose'])):
    df2['purpose_tex'].iloc[i]=' '.join(re.findall(r'[A-Za-z]+',df2['purpose'].iloc[i]))


0        OCT ADVANCE PAYM FOR THE INST RUMENTATIO N AND...
1                                            INVOICE No DD
2                                                  FEE USD
3                                            DOGOVOR ZAYMA
4                                            DOGOVOR ZAYMA
5                                            CONTRACT N DD
6                                            CONTRACT N DD
7                                               invoice dd
8                                               INVOICE DD
9                                            DOGOVOR ZAYMA
10                                           DOGOVOR ZAYMA
11                                              INVOICE DD
12                                              INVOICE DD
13                                              INVOICE DD
14                                      AHMADOV ANAR SUBAR
15                                              INVOICE DD
16                                              INVOICE 

In [275]:
df2['payer_account']=df2['payer_account'].fillna('Invalid')

for i in ['payer_bank_country','beneficiary_account','beneficiary_bank_country']:
    df2[i]=df2[i].fillna('Invalid')

In [331]:
df3=df2.drop(['source','purpose'],axis=1)
#source meaningless
df3['beneficiary_name_norm'].str.replace('.','')

0                           LCM ALLIANCE LLP
1                                 MOBILA LLP
2                        METASTAR INVEST LLP
3                        INMAXO CAPITAL CORP
4                        INMAXO CAPITAL CORP
5                        METASTAR INVEST LLP
6                        METASTAR INVEST LLP
7                  JETFIELD NETWORKS LIMITED
8                                 ERREBI SPA
9                             LOTA SALES LLP
10                            LOTA SALES LLP
11                        ELEATEX TRADING SL
12            CST KIMYA SANAYI VE TICARET AS
13                      SILVEA SCHIATTI+CSRL
14               EGE KURKAMU FINVE TICMRKZSB
15                        ELEATEX TRADING SL
16        UNIVAR KIMYA SANAYI VE DIS TIC LTD
17               MERTIK MAXITROL GMBH & COKG
18                      LINSTAR SYSTEMS CORP
19                       METASTAR INVEST LLP
20                       METASTAR INVEST LLP
21                       METASTAR INVEST LLP
22        

In [281]:
print(df3['beneficiary_name_norm'].value_counts())
df3['payer_name_norm_new'].value_counts()


LCM ALLIANCE LLP                      1758
METASTAR INVEST LLP                   1237
HILUX SERVICES LP                      905
POLUX MANAGEMENT LP                    433
FABERLEX LP                            334
KG COMMERCE LLP                        273
LOTA SALES LLP                         246
BONINVEST LLP                          234
CITYPORT CONTRACTS L.P.                214
INFOCREST LLP                          201
RIVERLANE LLP                          184
GFG EXPORT LLP                         178
REDPARK SALES CORP.                    140
DATEMILE ALLIANCE LLP                  117
MOLONEY TRADE LLP                      115
BONDWEST LLP                           113
AVROMED COMPANY LLP                    102
RICHFIELD TRADING L.P.                 101
UPFIELD EXPRESS INC                     95
SMART TEAM GENERAL TRADING LLC          91
GLOBECOM TRADE L.P.                     86
OVERMOND LLP                            83
RASMUS LP                               82
HAZAR INSAA

HILUX SERVICES LP                    5178
METASTAR INVEST LLP                  3846
LCM ALLIANCE LLP                     2820
POLUX MANAGEMENT LP                  1344
BAKTELEKOM MMC                        530
FABERLEX LP                           220
RIVERLANE LLP                         158
LOTA SALES LLP                        153
JETFIELD NETWORKS LIMITED             125
TOO ARMAS GROUP                        95
SOVER DEVELOPMENT LLP                  92
EVERMAIR PROJECTS LP                   84
BONINVEST LLP                          78
SOLIDSTAR WORLDWIDE LLP                63
LARKSTONE LIMITED                      61
JSC ROSOBORONEXPORT                    56
DELFRONT IMPORT LLP                    51
MOLONEY TRADE LLP                      46
INFOCREST LLP                          45
GLOBECOM TRADE LP                      45
KG COMMERCE LLP                        40
HIGH-RIDGE INVEST LTD                  38
TOO 'KAZANERDJISERVISEZ'               38
DELOKALI LIMITED                  

In [296]:
shell_ben_index=df3[(df3['beneficiary_name_norm']=='HILUX SERVICES LP') | (df3['beneficiary_name_norm']=='METASTAR INVEST LLP') |
   (df3['beneficiary_name_norm']=='POLUX MANAGEMENT LP') | (df3['beneficiary_name_norm']=='LCM ALLIANCE LLP')].index.values.tolist()

In [297]:
shell_payer_index=df3[(df3['payer_name_norm_new']=='HILUX SERVICES LP') | (df3['payer_name_norm_new']=='METASTAR INVEST LLP') |
   (df3['payer_name_norm_new']=='POLUX MANAGEMENT LP') | (df3['payer_name_norm_new']=='LCM ALLIANCE LLP')].index.values.tolist()

In [482]:
! pip install openpyxl

    100% |████████████████████████████████| 174kB 7.5MB/s ta 0:00:01
  Stored in directory: /Users/siruozhang/Library/Caches/pip/wheels/51/fa/dc/7eb403214c40a01befafce3ec0d215176d451d106a431d6781
  Stored in directory: /Users/siruozhang/Library/Caches/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile
You are using pip version 19.0.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [468]:
intersection_index_shell=[]
for m in shell_ben_index:
    if m in shell_payer_index:
        intersection_index_shell.append(m)
intersection_index_shell
df4=df3.iloc[intersection_index_shell]
#all transaction with core companies mutually

In [520]:
df3['purpose_text'].iloc[0]

['OCT',
 'ADVANCE',
 'PAYM',
 'FOR',
 'THE',
 'INST',
 'RUMENTATIO',
 'N',
 'AND',
 'CONTROL',
 'CABLES',
 'BOM',
 'IN',
 'ACC',
 'WIT',
 'H',
 'THE',
 'CON',
 'TR',
 'DD']

In [485]:

df3.to_excel('/Users/siruozhang/Desktop/clean_bootcamp.xlsx')

In [599]:
#re-clean purpose again
df3['purpose_text']=df2['purpose_text'].str.lower()
for i in range(len(df3['purpose_text'])):
    sentence_sent= df3['purpose_text'].iloc[i]
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence_sent)
    df3['purpose_text'].iloc[i] = [w for w in word_tokens if not w in stop_words]
df3['purpose_text']

0        [oct, advance, paym, inst, rumentatio, n, cont...
1                                            [invoice, dd]
2                                               [fee, usd]
3                                         [dogovor, zayma]
4                                         [dogovor, zayma]
5                                        [contract, n, dd]
6                                        [contract, n, dd]
7                                            [invoice, dd]
8                                            [invoice, dd]
9                                         [dogovor, zayma]
10                                        [dogovor, zayma]
11                                           [invoice, dd]
12                                           [invoice, dd]
13                                           [invoice, dd]
14                                  [ahmadov, anar, subar]
15                                           [invoice, dd]
16                                           [invoice, d

In [600]:
words_list=[]
for i in range(len(df3['purpose_text'])):
    for m in range(len(df3['purpose_text'].iloc[i])):
        words_list.append((df3['purpose_text'].iloc[i][m]))


    


In [608]:
from collections import Counter
words_count = Counter(words_list)
most_words = words_count.most_common(50)
most_words = [words for words in most_words if words[0] not in 'dd' 'n''e''c''r''f''l''b']
most_words


[('payment', 7896),
 ('invoice', 7734),
 ('contract', 5819),
 ('equipment', 1859),
 ('construction', 1292),
 ('materials', 1120),
 ('fee', 835),
 ('inv', 783),
 ('usd', 726),
 ('goods', 392),
 ('dogovor', 329),
 ('zayma', 327),
 ('material', 313),
 ('pment', 266),
 ('cntr', 229),
 ('service', 219),
 ('rfb', 212),
 ('bank', 203),
 ('az', 195),
 ('estonia', 195),
 ('danske', 193),
 ('date', 189),
 ('tallinn', 189),
 ('equi', 163),
 ('pmt', 161),
 ('truction', 160),
 ('contrakt', 160),
 ('p', 157),
 ('transportation', 152),
 ('swf', 142),
 ('ds', 140),
 ('contractno', 138),
 ('goo', 137),
 ('group', 135),
 ('textiles', 129),
 ('food', 125),
 ('agr', 123),
 ('vo', 122),
 ('bez', 121),
 ('nds', 121),
 ('services', 120),
 ('qui', 113)]

  Could not find a version that satisfies the requirement ImageColorGenerator (from versions: )
No matching distribution found for ImageColorGenerator
You are using pip version 19.0.2, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
